In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack, clone_model

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

In [5]:
query = """
select date(time), count(*) from point
where time > '2019-11-01'
group by 1
order by 1
"""
display(pd.read_sql_query(query, engine))

,date,count
0,2019-11-01,1
1,2019-11-02,2
2,2019-11-03,1
3,2019-11-04,1
4,2019-11-05,1
5,2019-11-06,2
6,2019-11-07,1
7,2019-11-09,3
8,2019-11-10,3
9,2019-11-11,1


# Find tracks for day

In [78]:
date = "2019-11-25"

In [79]:
bracket_query = """
select track.id, min(time) as min_time, max(time) as max_time from track
join point on track_id = track.id
group by track.id
having abs(date(min(time)) - %(date)s) < 2  or abs(date(max(time)) - %(date)s) < 2
"""
display(pd.read_sql_query(bracket_query, engine, params={'date': date}))


,id,min_time,max_time
0,273,2019-11-26 03:26:22+00:00,2019-11-26 03:26:22+00:00
1,176,2019-11-25 04:36:06+00:00,2019-11-25 04:36:53+00:00
2,173,2019-11-25 04:35:39+00:00,2019-11-25 04:35:39+00:00
3,10073,2019-11-26 03:42:47+00:00,2019-11-26 03:43:22+00:00
4,161,2019-11-25 03:44:18+00:00,2019-11-25 04:01:48+00:00
...,...,...,...
573,371,2019-11-26 09:12:38+00:00,2019-11-26 09:14:55+00:00
574,183,2019-11-25 06:30:30+00:00,2019-11-25 07:27:56+00:00
575,10144,2019-11-26 08:10:37+00:00,2019-11-26 08:12:14+00:00
576,122,2019-11-24 08:11:37+00:00,2019-11-24 08:11:37+00:00


In [80]:
bracket_tracks = [session.query(GPSTrack).get(int(t)) 
                  for t in pd.read_sql_query(bracket_query, engine, params={'date': date})[['id']].values]
len(bracket_tracks)

578

In [81]:
day_tracks = [t for t in bracket_tracks 
              if str(t.start.localtime.date()) == date or str(t.end.localtime.date()) == date]
len(day_tracks)

101

In [83]:
set([t.filename for t in day_tracks])

{'2019-11-25.gpx', None, 'inreach.gpx'}

In [84]:
set([t.name for t in day_tracks])

{'Aneel Nazareth (9914941)',
 'Ginza Station (銀座駅)',
 'Higashi-ginza Station (東銀座駅)',
 'Hotel Intergate Tokyo Kyobashi (ホテルインターゲート東京 京橋)',
 'JR新大阪駅 アルデ新大阪改札口',
 'Koishikawa Korakuen Garden (小石川後楽園)',
 'Marunouchi Line Tokyo Station (M17) (丸ノ内線 東京駅)',
 'Metro M Korakuen (メトロ・エム後楽園)',
 'Namba Station (なんば駅)',
 None,
 'SARASA HOTEL 日本橋',
 'Shinkansen Shin-Ōsaka Station (東海道・山陽新幹線 新大阪駅)',
 'Tokaido Shinkansen Tokyo Station (東海道新幹線 東京駅)',
 'Tsukiji Outer Market (築地場外市場)',
 'Tsukiji Station (H10) (築地駅)',
 'Tsurutontan (つるとんたん 宗右衛門町店)'}

In [85]:
set([t.start.localtime.date() for t in day_tracks])

{datetime.date(2019, 11, 25)}

# Visualization

In [86]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,
    Polyline, 
    Popup,
    GeoJSON,
    DrawControl
)
from ipywidgets import HTML

from traitlets import link

In [87]:
def marker(t):
    marker = Marker(location=t.points[0].lat_lon)
    marker.popup = HTML(f"{t.id}: {t.name}")
    return marker

def add_track(m, t, **kwargs):
        l = t.as_polyline(
            fill=False,
            **kwargs
        )
        m.add_layer(l)
        l.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{t.points[0].time} - {t.points[-1].time}")

In [88]:
b = list(day_tracks[0].bounds)
for t in day_tracks:
    tb = t.bounds
    b[0] = min(b[0], tb[0])
    b[1] = min(b[1], tb[1])
    b[2] = max(b[2], tb[2])
    b[3] = max(b[3], tb[3])
    
center = (b[1] + b[3]/2, (b[0] + b[2])/2)
b

[135.49844, 34.66229, 139.77328966690663, 35.707463]

In [89]:
m = Map(center=center, zoom=2, close_popup_on_click=False)

markers = []
for track in day_tracks:
    if len(track.points) == 1 and track.name is not None:
        markers.append(marker(track))
    else:
        add_track(m, track)
m.add_layer(MarkerCluster(markers=markers))

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Cleanup overlap

In [90]:
from intervaltree import IntervalTree
from datetime import timedelta

In [91]:
intervals = IntervalTree()
for t in day_tracks:
    start = t.start.time
    end = t.end.time
    if start == end:
        end += timedelta(seconds=1)
    intervals.addi(start, end, [t])
intervals.split_overlaps()
intervals.merge_equals(data_reducer=lambda a, b: a+b)

In [92]:
for i in sorted(intervals):
    print(f"{str(i.begin)}\t{str(i.end)}\t{i.end - i.begin}\t{' '.join(set([t.name or t.filename for t in i.data]))}")

2019-11-24 22:41:29+00:00	2019-11-24 22:41:30+00:00	0:00:01	Hotel Intergate Tokyo Kyobashi (ホテルインターゲート東京 京橋)
2019-11-24 23:33:45+00:00	2019-11-24 23:34:34+00:00	0:00:49	Aneel Nazareth (9914941)
2019-11-24 23:34:34+00:00	2019-11-24 23:34:35+00:00	0:00:01	Marunouchi Line Tokyo Station (M17) (丸ノ内線 東京駅) Aneel Nazareth (9914941)
2019-11-24 23:34:35+00:00	2019-11-24 23:55:23+00:00	0:20:48	Aneel Nazareth (9914941)
2019-11-24 23:55:23+00:00	2019-11-25 00:01:03+00:00	0:05:40	Aneel Nazareth (9914941) 2019-11-25.gpx
2019-11-25 00:01:03+00:00	2019-11-25 00:01:04+00:00	0:00:01	Marunouchi Line Tokyo Station (M17) (丸ノ内線 東京駅) Aneel Nazareth (9914941) 2019-11-25.gpx
2019-11-25 00:01:04+00:00	2019-11-25 00:05:13+00:00	0:04:09	Aneel Nazareth (9914941) 2019-11-25.gpx
2019-11-25 00:05:13+00:00	2019-11-25 00:05:14+00:00	0:00:01	Ginza Station (銀座駅) Aneel Nazareth (9914941) 2019-11-25.gpx
2019-11-25 00:05:14+00:00	2019-11-25 00:08:50+00:00	0:03:36	Aneel Nazareth (9914941) 2019-11-25.gpx
2019-11-25 00:08:50+00

# Output

In [93]:
import gpxpy

In [94]:
gpx = gpxpy.gpx.GPX()
for i in sorted(intervals):
    track_ids = [t.id for t in i.data]
    points = (
        session.query(GPSPoint)
        .filter(GPSPoint.track_id.in_(track_ids))
        .filter(GPSPoint.time.between(i.begin, i.end))
        .order_by(GPSPoint.time)
    ).all()
    if not points:
        continue
    
    if len(points) == 1:
        p = points[0]
        t = p.track
        w = gpxpy.gpx.GPXWaypoint(
            latitude=p.latitude, longitude=p.longitude, elevation=p.elevation,
            time=p.time, name=t.name, description=t.description, type=t.type,
            comment=t.source or t.filename
        )
        gpx.waypoints.append(w)
    else:
        t = gpxpy.gpx.GPXTrack(
            name=f"{points[0].localtime.time()} {' '.join({t.type or '' for t in i.data})}",
            description="\t".join({t.description or "" for t in i.data}),
        )
        t.comment="\t".join({t.name or t.filename or "" for t in i.data})
        t.source="\t".join({t.source or "" for t in i.data})
        t.type="\t".join({t.type or "" for t in i.data})
        s = gpxpy.gpx.GPXTrackSegment()
        t.segments.append(s)
        for p in points:
            s.points.append(gpxpy.gpx.GPXTrackPoint(
                latitude=p.latitude, longitude=p.longitude, elevation=p.elevation, time=p.time))
        gpx.tracks.append(t)
    print(f"{str(points[0].localtime)}\t{str(points[-1].localtime.time())}\t{i.end - i.begin}\t{len(points)}\t{set(t.name or t.filename for t in i.data)}")
    
    

2019-11-25 07:41:29+09:00	07:41:29	0:00:01	1	{'Hotel Intergate Tokyo Kyobashi (ホテルインターゲート東京 京橋)'}
2019-11-25 08:33:45+09:00	08:34:28	0:00:49	12	{'Aneel Nazareth (9914941)'}
2019-11-25 08:34:34+09:00	08:34:34	0:00:01	1	{'Marunouchi Line Tokyo Station (M17) (丸ノ内線 東京駅)', 'Aneel Nazareth (9914941)'}
2019-11-25 08:34:38+09:00	08:45:29	0:20:48	92	{'Aneel Nazareth (9914941)'}
2019-11-25 08:55:23+09:00	09:01:01	0:05:40	50	{'Aneel Nazareth (9914941)', '2019-11-25.gpx'}
2019-11-25 09:01:03+09:00	09:01:03	0:00:01	1	{'Marunouchi Line Tokyo Station (M17) (丸ノ内線 東京駅)', 'Aneel Nazareth (9914941)', '2019-11-25.gpx'}
2019-11-25 09:01:06+09:00	09:05:02	0:04:09	26	{'Aneel Nazareth (9914941)', '2019-11-25.gpx'}
2019-11-25 09:05:13+09:00	09:05:13	0:00:01	1	{'Ginza Station (銀座駅)', 'Aneel Nazareth (9914941)', '2019-11-25.gpx'}
2019-11-25 09:05:20+09:00	09:08:50	0:03:36	32	{'Aneel Nazareth (9914941)', '2019-11-25.gpx'}
2019-11-25 09:08:50+09:00	09:08:50	0:00:01	3	{'Aneel Nazareth (9914941)', '2019-11-25.gpx', 

In [95]:
with open(f"{date}.gpx", "w") as fh:
    print(gpx.to_xml(version="1.1"), file=fh)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …